<a href="https://colab.research.google.com/github/AhmedAlhisan/FrudDetection/blob/main/fine_tune_whesper_using_Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate==1.7.0 \
             bitsandbytes==0.46.0 \
             datasets==2.21.0 \
             evaluate==0.4.3 \
             huggingface-hub==0.33.0 \
             loralib==0.1.2 \
             transformers==4.52.4 \
             ctranslate2==4.6.0 \
             sentencepiece==0.2.0 \
             librosa==0.11.0 \
             soundfile==0.13.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.8/514.8 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install git+https://github.com/huggingface/peft.git@759bb70acea8486b3a493792a5b6ba80b5217fb4


  Cloning https://github.com/huggingface/peft.git (to revision 759bb70acea8486b3a493792a5b6ba80b5217fb4) to /tmp/pip-req-build-xe4je729
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-xe4je729
  Running command git rev-parse -q --verify 'sha^759bb70acea8486b3a493792a5b6ba80b5217fb4'
  Running command git fetch -q https://github.com/huggingface/peft.git 759bb70acea8486b3a493792a5b6ba80b5217fb4
  Resolved https://github.com/huggingface/peft.git to commit 759bb70acea8486b3a493792a5b6ba80b5217fb4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.15.2.dev0-py3-none-any.whl size=440235 sha256=1cc0b6f8165703d0b883e02386955f5b2e2cafbb78f33acf2b18c16c7a651922
  Stored in directory: /root/.cache/pip/wheels/6c/04/94/f03a1fb6fa6bdd47dd1c8359143f9b16d3890a1e14376d7a04
Successfully built peft
  Attempting unin

In [3]:
import subprocess, sys, torch, platform
print("Python  :", platform.python_version())
print("Torch   :", torch.__version__)
print("CUDA OK :", torch.cuda.is_available())
print("Device  :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "-")


Python  : 3.11.13
Torch   : 2.6.0+cu124
CUDA OK : True
Device  : NVIDIA A100-SXM4-40GB


In [4]:

import torch, platform
print("Python :", platform.python_version())
print("Torch  :", torch.__version__)
print("CUDA OK:", torch.cuda.is_available())


Python : 3.11.13
Torch  : 2.6.0+cu124
CUDA OK: True


In [5]:
# ---------- Step 2 (Turbo) ----------
from huggingface_hub import notebook_login
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import torch

# 1️⃣  Log in to Hugging Face (paste your token when prompted)
notebook_login()

# 2️⃣  Turbo checkpoint (4 decoder layers instead of 32 → much faster & lighter)
MODEL_NAME = "openai/whisper-large-v3-turbo"

# 3️⃣  Load processor + model on GPU in fp16
processor = AutoProcessor.from_pretrained(MODEL_NAME)

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

print("✓ model loaded:", MODEL_NAME)
print("Total parameters:",
      sum(p.numel() for p in model.parameters()) // 1_000_000, "M")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.71M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

✓ model loaded: openai/whisper-large-v3-turbo
Total parameters: 808 M


In [6]:
import torch, numpy as np, inspect
from typing import Any, Dict, List, Union
from dataclasses import dataclass
from transformers import WhisperFeatureExtractor, WhisperTokenizer
from peft import LoraConfig, get_peft_model, PeftModelForSeq2SeqLM
from types import MethodType


In [7]:
use_bf16   = False
input_dtype = torch.bfloat16 if use_bf16 else torch.float16
feature_extractor: WhisperFeatureExtractor = processor.feature_extractor
tokenizer: WhisperTokenizer               = processor.tokenizer
# ------------------------------------------------------------------

In [8]:
# 1️⃣  Map → add "input_features" & "labels"  (keep originals for now)
def prepare(batch):
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"]
    ).input_features[0]
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [9]:

# 3️⃣  Data collator (unchanged)
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    def __call__(self, feats):
        inputs = [{"input_features": f["input_features"]} for f in feats]
        batch  = self.processor.feature_extractor.pad(inputs, return_tensors="pt")
        batch  = {k: v.to(input_dtype) for k, v in batch.items()}

        labels_feats = [{"input_ids": f["labels"]} for f in feats]
        labels_batch = self.processor.tokenizer.pad(labels_feats, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [10]:
# 4️⃣  LoRA attach
lora_cfg = LoraConfig(
    r=32, lora_alpha=8, lora_dropout=0.05, use_rslora=True,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj", "fc1", "fc2"],
    bias="none", task_type="SEQ_2_SEQ_LM",
)
model.enable_input_require_grads()
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

trainable params: 27,852,800 || all params: 836,730,880 || trainable%: 3.3288


In [11]:
from types import MethodType

def _strip_input_ids(self, *args, **kw):
    kw.pop("input_ids", None)          # drop the obsolete kwarg
    return self.base_model(*args, **kw)


model.forward = MethodType(_strip_input_ids, model)  # <-- patch ONLY this instance




In [12]:
import peft, transformers
print("peft        :", peft.__version__)
print("transformers:", transformers.__version__)


peft        : 0.15.2.dev0
transformers: 4.52.4


In [13]:
from datasets import load_dataset, Audio

RAW = "MohamedRashad/SADA22"

# ➊ load the split(s) you need
ds = load_dataset(RAW, split="train")

# ➋ let 🤗 cast the audio so we get "array" + "sampling_rate"
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

# ➌ keep only clips ≤ 30 s  (30 000 ms == 30 s at 16 kHz ⇒ 480 000 samples)
MAX_SAMPLES = 30 * 16_000        # 480 000
ds = ds.filter(lambda x: len(x["audio"]["array"]) <= MAX_SAMPLES)

print("✓ after filter:", ds.num_rows, "clips")


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/241834 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5139 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6193 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/95 [00:00<?, ?it/s]

Filter:   0%|          | 0/241834 [00:00<?, ? examples/s]

✓ after filter: 236191 clips


In [14]:




ds_val = load_dataset(RAW, split="validation")

ds_val = ds_val.cast_column("audio", Audio(sampling_rate=16_000))

MAX_SAMPLES = 30 * 16_000        # 480 000
ds_val = ds_val.filter(lambda x: len(x["audio"]["array"]) <= MAX_SAMPLES)

print("after filter:", ds_val.num_rows, "clips")


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Filter:   0%|          | 0/5139 [00:00<?, ? examples/s]

after filter: 5029 clips


In [23]:
import random
from datasets import Dataset
SAMPLE_SIZE = 8000

# 1️⃣  make sure we actually have that many clips
assert len(ds) >= SAMPLE_SIZE, f"Only {len(ds)} clips available!"

# 2️⃣  reproducible random sample of indices
rng = random.Random(42)                 # pick any seed you like
indices = rng.sample(range(len(ds)), SAMPLE_SIZE)

sample_ds = ds.select(indices)

print("sample size:", sample_ds.num_rows)
print(sample_ds[0])                     # quick check


sample size: 8000
{'audio': {'path': 'c09a506a-8320-4783-91a4-18c9e31826ea.wav', 'array': array([ 0.19024658,  0.11703491,  0.04501343, ..., -0.02783203,
       -0.02868652, -0.027771  ]), 'sampling_rate': 16000}, 'text': 'ولو جا يوم وفهمتيني', 'cleaned_text': 'ولو جا يوم وفهمتيني', 'speaker_age': 'Unknown', 'speaker_gender': 'Unknown', 'speaker_dialect': 'Unknown'}


In [24]:

from transformers import WhisperFeatureExtractor, WhisperTokenizer
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

feature_extractor: WhisperFeatureExtractor = processor.feature_extractor
tokenizer:           WhisperTokenizer      = processor.tokenizer
use_bf16   = False
input_dtype = torch.bfloat16 if use_bf16 else torch.float16

def prepare(batch):
    # audio → log-Mel
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"]
    ).input_features[0]
    # cleaned_text → token IDs
    batch["labels"] = tokenizer(batch["cleaned_text"]).input_ids
    return batch

ds_proc_train = sample_ds.map(prepare, remove_columns=sample_ds.column_names, num_proc=1)
print("Ready to train", ds_proc_train.column_names)

ds_proc_validation = ds_val.map(prepare, remove_columns=ds_val.column_names, num_proc=1)
print("Ready to validation", ds_proc_train.column_names)


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Ready to train ['input_features', 'labels']
Ready to validation ['input_features', 'labels']


In [25]:

from datasets import DatasetDict


train_ds = ds_proc_train
val_ds   = ds_proc_validation

ds_final = DatasetDict({"train": train_ds, "validation": val_ds})
print(ds_final)


DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 5029
    })
})


In [26]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    def __call__(self, feats: List[Dict[str, Union[List[int], torch.Tensor]]]):
        inp = [{"input_features": f["input_features"]} for f in feats]
        batch = self.processor.feature_extractor.pad(inp, return_tensors="pt")
        batch = {k: v.to(input_dtype) for k, v in batch.items()}

        labs = [{"input_ids": f["labels"]} for f in feats]
        labs_batch = self.processor.tokenizer.pad(labs, return_tensors="pt")
        labels = labs_batch["input_ids"].masked_fill(
            labs_batch.attention_mask.ne(1), -100
        )
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer,EarlyStoppingCallback

OUTPUT_DIR = "whisper_turbo_sa_lora_full"

training_args = Seq2SeqTrainingArguments(
    output_dir                 = OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_train_epochs           = 2,
    learning_rate              = 1e-4,
    warmup_steps               = 50,
    fp16                       = True,
    logging_steps              = 10,
    eval_strategy              = "steps",
    eval_steps                 = 100,
    save_strategy              = "steps",
    save_steps                 = 50,
    save_total_limit           = 3,
    load_best_model_at_end     = True,
    predict_with_generate      = True,
    generation_max_length      = 128,
    do_eval=True,

    dataloader_num_workers     = 0,
    report_to                  = "none",
)

trainer = Seq2SeqTrainer(
    model         = model,
    args          = training_args,
    train_dataset = ds_final["train"],
    eval_dataset  = ds_final["validation"],
    data_collator = data_collator,

)
trainer.train()


No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
50,0.532600,0.804238
100,1.121800,0.787084
150,0.702900,0.749592


Step,Training Loss,Validation Loss
50,0.532600,0.804238
100,1.121800,0.787084
150,0.702900,0.749592
200,1.183300,0.793723
250,1.110600,0.746232
300,0.521900,0.760054
